In [4]:
!pip install pathway bokeh panel geopy --quiet


In [5]:
import numpy as np
import pandas as pd
import datetime
from geopy.distance import geodesic
import pathway as pw
import bokeh.plotting as bk
import panel as pn
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show


In [6]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset (1).csv


In [7]:
df = pd.read_csv('dataset.csv')  # Upload manually in Colab
df['timestamp'] = pd.to_datetime(df['LastUpdatedTime'])
df.sort_values(by='timestamp', inplace=True)
df.head()


/tmp/ipython-input-7-3015778637.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['LastUpdatedTime'])


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,timestamp
7370,7370,BHMNCPNST01,485,26.140048,91.730972,215,car,low,2,1,20-11-2016,07:41:00,2025-07-09 07:41:00
7518,7518,BHMNCPNST01,485,26.140048,91.730972,193,bike,low,1,0,28-11-2016,07:42:00,2025-07-09 07:42:00
7514,7514,BHMNCPNST01,485,26.140048,91.730972,193,car,low,1,0,28-11-2016,07:42:00,2025-07-09 07:42:00
7515,7515,BHMNCPNST01,485,26.140048,91.730972,193,truck,low,2,0,28-11-2016,07:42:00,2025-07-09 07:42:00
7516,7516,BHMNCPNST01,485,26.140048,91.730972,193,car,low,2,0,28-11-2016,07:42:00,2025-07-09 07:42:00


In [24]:
df = df.rename(columns={
    'ID': 'lot_id',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'Occupancy': 'occupancy',
    'Capacity': 'capacity',
    'QueueLength': 'queue_length',
    'TrafficConditionNearby': 'traffic_level',
    'IsSpecialDay': 'special_day',
    'VehicleType': 'vehicle_type'
})


In [25]:
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['vehicle_type'] = df['vehicle_type'].map(vehicle_map)


In [26]:
def model1_linear_price(prev_price, occupancy, capacity, alpha=2):
    occ_rate = occupancy / capacity
    new_price = prev_price + alpha * occ_rate
    return max(5, min(20, new_price))


In [27]:
def model2_demand_price(base_price, row, weights, lambd=0.2):
    occ = row['occupancy']
    cap = row['capacity']
    queue = row['queue_length']
    traffic = row['traffic_level']
    is_special = row['special_day']
    vehicle_type = row['vehicle_type']
    demand = (
        weights['occ'] * (occ / cap) +
        weights['queue'] * queue -
        weights['traffic'] * traffic +
        weights['special'] * is_special +
        weights['vehicle'] * vehicle_type
    )
    norm_demand = (demand - 0) / 10
    price = base_price * (1 + lambd * norm_demand)
    return max(5, min(20, price))


In [28]:
def get_nearby_lots(current_lot_id, df_meta, radius_m=500):
    current = df_meta[df_meta['lot_id'] == current_lot_id].iloc[0]
    current_coords = (current['latitude'], current['longitude'])
    nearby = []
    for _, row in df_meta.iterrows():
        if row['lot_id'] == current_lot_id:
            continue
        other_coords = (row['latitude'], row['longitude'])
        distance = geodesic(current_coords, other_coords).meters
        if distance <= radius_m:
            nearby.append(row['lot_id'])
    return nearby


In [29]:
def model3_competitive_price(base_price, row, current_time, all_prices_df, df_meta, weights, lambd=0.2):
    occ = row['occupancy']
    cap = row['capacity']
    queue = row['queue_length']
    traffic = row['traffic_level']
    is_special = row['special_day']
    vehicle_type = row['vehicle_type']
    lot_id = row['lot_id']
    demand = (
        weights['occ'] * (occ / cap) +
        weights['queue'] * queue -
        weights['traffic'] * traffic +
        weights['special'] * is_special +
        weights['vehicle'] * vehicle_type
    )
    norm_demand = (demand - 0) / 10
    my_price = base_price * (1 + lambd * norm_demand)
    nearby_lots = get_nearby_lots(lot_id, df_meta)
    recent_prices = all_prices_df[
        (all_prices_df['timestamp'] == current_time) &
        (all_prices_df['lot_id'].isin(nearby_lots))
    ]
    if not recent_prices.empty:
        avg_comp_price = recent_prices['price'].mean()
        if occ >= cap and my_price > avg_comp_price:
            my_price = max(base_price, avg_comp_price - 1)
        elif avg_comp_price > my_price:
            my_price = min(my_price + 1, 20)
    return max(5, min(20, my_price))


In [13]:
def simulate_model3(df, df_meta, base_price=10):
    df = df.copy()
    df.sort_values(by='timestamp', inplace=True)
    prices = []
    all_prices_df = pd.DataFrame(columns=['timestamp', 'lot_id', 'price'])
    weights = {
        'occ': 1,
        'queue': 0.5,
        'traffic': 0.7,
        'special': 0.8,
        'vehicle': 0.4
    }
    for idx, row in df.iterrows():
        current_time = row['timestamp']
        lot_id = row['lot_id']
        price = model3_competitive_price(
            base_price, row, current_time,
            all_prices_df, df_meta, weights
        )
        prices.append(price)
        all_prices_df = pd.concat([
            all_prices_df,
            pd.DataFrame([{'timestamp': current_time, 'lot_id': lot_id, 'price': price}])
        ])
    df['price'] = prices
    return df


In [30]:
def plot_prices(df, lot_id_filter):
    filtered = df[df['lot_id'] == lot_id_filter]
    source = ColumnDataSource(filtered)
    p = figure(x_axis_type='datetime', title=f'Price Over Time – Lot {lot_id_filter}')
    p.line(x='timestamp', y='price', source=source, line_width=2)
    show(p)


In [22]:
print(df.columns.tolist())


['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'timestamp']


In [37]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

def plot_prices(df, lot_id_filter):
    filtered = df[df['lot_id'] == lot_id_filter]
    source = ColumnDataSource(filtered)

    p = figure(x_axis_type='datetime', title=f'Price Over Time – Lot {lot_id_filter}')
    p.line(x='timestamp', y='price', source=source, line_width=2)
    show(p)


In [38]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()  # for displaying in Jupyter/Colab

def plot_prices(df, lot_id_filter):
    filtered = df[df['lot_id'] == lot_id_filter]
    source = ColumnDataSource(filtered)

    p = figure(x_axis_type='datetime',
               title=f'📈 Price Over Time – Lot {lot_id_filter}',
               width=800, height=400)

    p.line(x='timestamp', y='price', source=source, line_width=2, color="navy")
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    p.grid.grid_line_alpha = 0.3

    show(p)


In [39]:
from bokeh.io import output_notebook, show
output_notebook()


In [40]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()  # for displaying in Jupyter/Colab

def plot_prices(df, lot_id_filter):
    filtered = df[df['lot_id'] == lot_id_filter]
    source = ColumnDataSource(filtered)

    p = figure(x_axis_type='datetime',
               title=f'📈 Price Over Time – Lot {lot_id_filter}',
               width=800, height=400)

    p.line(x='timestamp', y='price', source=source, line_width=2, color="navy")
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    p.grid.grid_line_alpha = 0.3

    show(p)

NameError: name 'sim_df' is not defined